In [4]:
from glob import glob
import os

files = glob('mysoalan.com-pdf/*.pdf')
files = sorted(list(set(files)))
len(files)

1672

In [5]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup
from glob import glob

In [6]:
TIKA_HOST = 'http://localhost:9998'

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()


def parse_tika(file, minlen=2):
    raw_xml = parser.from_file(file, TIKA_HOST, xmlContent=True, requestOptions = {'timeout': 3600})
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass
    
    return t

In [7]:
with open('mysoalan.com-tika.jsonl', 'w') as fopen:
    for f in tqdm(files):
        try:
            t = '\n'.join(parse_tika(f))
            data = {
                'file': f,
                'text': t,
            }
            fopen.write(f'{json.dumps(data)}\n')
        except Exception as e:
            print(e)

  0%|                                                                                        | 0/1672 [00:00<?, ?it/s]/tmp/ipykernel_1942971/3636658005.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = soup.findAll(text=True)
100%|█████████████████████████████████████████████████████████████████████████████| 1672/1672 [01:12<00:00, 23.15it/s]


In [8]:
from huggingface_hub import HfApi
api = HfApi()

In [9]:
api.upload_file(
    path_or_fileobj='mysoalan.com-tika.jsonl',
    path_in_repo='mysoalan.com-tika.jsonl',
    repo_id='malaysia-ai/mysoalan.com',
    repo_type='dataset',
)

mysoalan.com-tika.jsonl:   0%|          | 0.00/22.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/mysoalan.com/commit/9453f4ab1d52368a65830cc4aa028e266107b94e', commit_message='Upload mysoalan.com-tika.jsonl with huggingface_hub', commit_description='', oid='9453f4ab1d52368a65830cc4aa028e266107b94e', pr_url=None, pr_revision=None, pr_num=None)